## Luftqualität in Hamburg via AQICN (Air Quality Index)

token=5a018480b553ee583496128fe95decf8583e0fae

In [ ]:
import requests
import pandas as pd
from datetime import datetime

API_TOKEN = "5a018480b553ee583496128fe95decf8583e0fae"
BASE_API_URL = "https://api.waqi.info/feed/{station}/?token={token}"
CSV_FILE = "aqi_hamburg_all_stations.csv"

# Liste der Hamburger Messstationen (Beispiel)
stations = [""]

def fetch_data_for_all_stations():
    all_results = []

    for station in stations:
        api_url = BASE_API_URL.format(station=station, token=API_TOKEN)
        response = requests.get(api_url)
        data = response.json()

        if data["status"] != "ok":
            print(f"Fehlerhafte API-Antwort für Station {station}.")
            continue

        d = data["data"]
        result = {
            "station": station,
            "timestamp": pd.to_datetime(d["time"]["iso"]),
            "aqi": d.get("aqi"),
            "pm25": d.get("iaqi", {}).get("pm25", {}).get("v"),
            "pm10": d.get("iaqi", {}).get("pm10", {}).get("v"),
            "no2": d.get("iaqi", {}).get("no2", {}).get("v"),
            "co": d.get("iaqi", {}).get("co", {}).get("v"),
            "o3": d.get("iaqi", {}).get("o3", {}).get("v")
        }
        all_results.append(result)

    df = pd.DataFrame(all_results)
    try:
        old_df = pd.read_csv(CSV_FILE)
        df = pd.concat([old_df, df], ignore_index=True)
        df.drop_duplicates(subset=["station", "timestamp"], inplace=True)
    except FileNotFoundError:
        pass

    df.to_csv(CSV_FILE, index=False)
    print(f"✅ Daten für alle Stationen gespeichert.")

if __name__ == "__main__":
    fetch_data_for_all_stations()

Fehlerhafte API-Antwort für Station hamburg/altona.
Fehlerhafte API-Antwort für Station hamburg/blankenese.
Fehlerhafte API-Antwort für Station hamburg/sternschanze.
Fehlerhafte API-Antwort für Station hamburg/bergedorf.
✅ Daten für alle Stationen gespeichert.
